## 📌 Overview

#### 📋 Key Aspects of the Notebook
This notebook is used for the inference of three pretrained models, ResNet34d, EfficientNetB0 and EfficientnetB1: trained exclusively on the Kaggle-provided data (EEG spectrograms) using a batch size of 16. 
* links:
    * https://www.kaggle.com/code/andreasbis/hms-train-resnet34d
    * https://www.kaggle.com/code/andreasbis/hms-train-efficientnetb0
    * https://www.kaggle.com/code/andreasbis/hms-train-efficientnetb1


### 🙏 Acknowledgement
Inspired by the work of @yunsuxiaozi. Don't forget to upvote their work if you find it helpful!
* links:
    * https://www.kaggle.com/code/yunsuxiaozi/hms-baseline-resnet34d-512-512-training-5-folds
    * https://www.kaggle.com/code/yunsuxiaozi/hms-baseline-resnet34d-512-512-inference-6-models

## 📙 Import libraries and modules

In [1]:
# Importing essential libraries
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display

# PyTorch for deep learning
import timm
import torch
import torch.nn as nn  
import torch.optim as optim
import torch.nn.functional as F

# torchvision for image processing and augmentation
import torchvision.transforms as transforms

# Suppressing minor warnings to keep the output clean
warnings.filterwarnings('ignore', category=Warning)

# Reclaim memory no longer in use.
gc.collect()

46

## ⚙️ Configuration

In [2]:
class Config:
    seed=42
    image_transform=transforms.Resize((512, 512))
    num_folds=5
    
# Set the seed for reproducibility across multiple libraries
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(Config.seed)

## 📂 Data Loading

In [3]:
models = []

# Update to include full paths for each architecture
arch_paths = {
    'resnet34d': '/kaggle/input/k/apatti01/hms-train-resnet34d',
    'efficientnet_b0': '/kaggle/input/k/apatti01/hms-train-efficientnetb0',
    'efficientnet_b1': '/kaggle/input/k/apatti01/hms-train-efficientnetb1'
}

def load_models(architecture, type_prefix, num_folds, base_path):
    for i in range(num_folds):
        model_path = f'{base_path}/{type_prefix}/{architecture}_fold{i}.pth'
        model = timm.create_model(architecture, pretrained=False, num_classes=6, in_chans=1)
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
        models.append(model)

for arch, base_path in arch_paths.items():
    # Load "less10" models
    load_models(arch, 'models_less10', Config.num_folds, base_path)
    # Load "more10" models
    load_models(arch, 'models_more10', Config.num_folds, base_path)

gc.collect()



36

In [4]:
# Load test data and sample submission dataframe
test_df = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")

# Merge the submission dataframe with the test data on EEG IDs
submission = submission.merge(test_df, on='eeg_id', how='left')

# Generate file paths for each spectrogram based on the EEG data in the submission dataframe
submission['path'] = submission['spectrogram_id'].apply(lambda x: f"/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/{x}.parquet")

# Display the first few rows of the submission dataframe
display(submission.head())

# Reclaim memory no longer in use
gc.collect()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,patient_id,path
0,3911565283,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,853520,6885,/kaggle/input/hms-harmful-brain-activity-class...


0

## 🎰 Predictions

In [5]:
# Define the weights for each model architecture
weight_resnet34d = 0.26
weight_effnetb0 = 0.48
weight_effnetb1 = 0.26

# Define weights for "less10" and "more10"
weight_less10 = 0.4
weight_more10 = 0.6

# Get file paths for test spectrograms
paths = submission['path'].values
test_preds = []

# Generate predictions for each spectrogram using all models
for path in paths:
    eps = 1e-6
    # Read and preprocess spectrogram data
    data = pd.read_parquet(path)
    data = data.fillna(-1).values[:, 1:].T
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)
    
    # Normalize the data
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    data = (data - data_mean) / (data_std + eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data = Config.image_transform(data_tensor)

    test_pred = []
    
    # Generate predictions using all models
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)), dim=1)[0]
            pred = pred.detach().cpu().numpy()
        test_pred.append(pred)
        
    # Separate predictions into "less10" and "more10" for each architecture
    preds_resnet34d = test_pred[:10]  # First 10 models are for ResNet34d
    preds_effnetb0 = test_pred[10:20]  # Next 10 models are for EfficientNetB0
    preds_effnetb1 = test_pred[20:]  # Last 10 models are for EfficientNetB1
    
    # Calculate weighted averages for "less10" and "more10"
    weighted_pred_resnet34d = np.mean(preds_resnet34d[:5], axis=0) * weight_less10 + np.mean(preds_resnet34d[5:], axis=0) * weight_more10
    weighted_pred_effnetb0 = np.mean(preds_effnetb0[:5], axis=0) * weight_less10 + np.mean(preds_effnetb0[5:], axis=0) * weight_more10
    weighted_pred_effnetb1 = np.mean(preds_effnetb1[:5], axis=0) * weight_less10 + np.mean(preds_effnetb1[5:], axis=0) * weight_more10
    
    # Combine weighted predictions from all architectures
    final_pred = (weighted_pred_resnet34d * weight_resnet34d +
                  weighted_pred_effnetb0 * weight_effnetb0 +
                  weighted_pred_effnetb1 * weight_effnetb1)
    
    test_preds.append(final_pred)

# Convert the list of predictions to a NumPy array for further processing
test_preds = np.array(test_preds)

# Reclaim memory no longer in use
gc.collect()


0

## 🚀 Submission

In [6]:
# Load the sample submission file and update it with model predictions for each label
submission = pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']

# Assign model predictions to respective columns in the submission DataFrame
for i in range(len(labels)):
    submission[f'{labels[i]}_vote'] = test_preds[:, i]

# Save the updated DataFrame as the final submission file
submission.to_csv("submission.csv", index=None)

# Display the first few rows of the submission file
display(submission.head())

# Reclaim memory no longer in use.
gc.collect()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,3911565283,0.081307,0.11103,0.001382,0.256873,0.022374,0.527034


0